<a href="https://colab.research.google.com/github/Krish-Roorkee/Sustainable-Delivery-Optimization-/blob/main/rx3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
!pip install pyswarms
import pyswarms as ps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.4 MB/s eta 0:00:00


In [3]:
nodes = 3;

def obj(WW, initPos, initTimeSpent, initLoading, distMat, reqLoads):
  load_status = np.zeros((nodes, 5));
  speed = 1000;
  charge_rate = 10;
  discharge_rate = 1/10000;
  cap = 5;
  Cl = 10;
  Cd = 1/100;
  initTimeSpent = np.abs(initTimeSpent);
  initLoading = initLoading.astype(int);
  cost = 0;
  for k in range(len(WW)):
    W = WW[k];
    checkpoints = np.abs(W[0]).astype(int);
    checkpoints = np.mod(checkpoints, 3);
    loading = W[1].astype(int);
    timeSpent = np.abs(W[2]);
    time = 0;
    distance = 0;
    load = 0;
    charge = 1;
    pos = initPos[k];
    if(int(time)<5):
      load_status[pos][int(time)] += initLoading[k];
      load -= initLoading[k];
    else:
      cost += 1000*(time-5);

    time += initTimeSpent[k];

    for n in range(len(checkpoints)):
      if(load>cap):
        cost += 10000*(load-cap);
      elif(load<0):
        cost -= 1000*load;
      distance += distMat[pos][checkpoints[n]];
      time += distMat[pos][checkpoints[n]]/speed;
      charge -= distMat[pos][checkpoints[n]]*discharge_rate;
      if(charge<0):
        cost -= 5000*charge;
      pos = checkpoints[n];
      if(int(time)<5):
        load_status[pos][int(time)] += loading[n];
        load -= loading[n];
      else:
        cost += 1000*(time-5);
      time += timeSpent[n];
      charge += timeSpent[n]*charge_rate;
    if(load>cap):
        cost += 10000*(load-cap);
    elif(load<0):
      cost -= 1000*load;
    cost += Cd*distance;
  cost += Cl*np.sum(np.square(reqLoads - load_status), axis=None);
  return cost;

In [ ]:
distMat = [[0, 100, 200],
           [100, 0, 150],
           [200, 150, 0]];
reqLoads = [[-10, 0, 0, 0, 0],
            [0, 0, 0, 2, 0],
            [0, 6, 0, 0, 2]];
initPos = [0, 2];
m = 2;
def g(w):
  initLoading = w[0:m];
  initTimeSpent = np.abs(w[m:2*m]);
  w = w[2*m:]
  w = np.reshape(w, (m, 3, 6));
  return obj(w, initPos, initTimeSpent, initLoading, distMat, reqLoads);




In [ ]:
def f(W):
  return np.array([g(w) for w in W]);

# Set-up hyperparameters
options = {'c1': 0.2, 'c2': 0.9, 'w': 0.9}

# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=1000, dimensions=m*18 + 2*m, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2023-11-30 10:37:39,748 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.2, 'c2': 0.9, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=9
2023-11-30 10:38:44,288 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 9.0, best pos: [ 0.40850534  0.4234294  -0.15952907  0.54368159 -0.18709118  0.90075834
  2.53491863 -5.33337061  2.81166859  2.30127427 -2.49518659 -3.53719267
  2.91444219  3.04805197 -0.34983784 -0.21479649  0.6991146  -0.27893253
  0.26154098  1.72180611  0.90913278 -4.7124667   0.3490396   0.30720375
  2.956625    1.37154268  1.09611679 -2.31166024 -2.38509514 -3.70594992
  1.3160427  -0.01646558  2.89251243  2.74667394  0.04220646 -0.26191177
 -0.03102778 -2.39303425 -0.36164273 -1.84518334]


In [ ]:
def objV(WW, initPos, initTimeSpent, initLoading, distMat, reqLoads):
  load_status = np.zeros((nodes, 5));
  speed = 1000;
  cap = 5;
  Cl = 1;
  Cd = 1/100;
  initTimeSpent = np.abs(initTimeSpent);
  initLoading = initLoading.astype(int);
  cost = 0;
  for k in range(len(WW)):
    W = WW[k];
    checkpoints = np.abs(W[0]).astype(int);
    checkpoints = np.mod(checkpoints, 3);
    WW[k][0] = checkpoints;
    loading = W[1].astype(int);
    WW[k][1] = loading;
    timeSpent = np.abs(W[2]);
    WW[k][2] = timeSpent;
    time = 0;
    distance = 0;
    load = 0;
    pos = initPos[k];
    if(int(time)<5):
      load_status[pos][int(time)] += initLoading[k];
      load -= initLoading[k];
    time += initTimeSpent[k];

    for n in range(len(checkpoints)):
      if(load>cap):
        cost += 10000*(load-cap);
      distance += distMat[pos][checkpoints[n]];
      time += distMat[pos][checkpoints[n]]/speed;
      pos = checkpoints[n];
      if(int(time)<5):
        load_status[pos][int(time)] += loading[n];
        load -= loading[n];
      time += timeSpent[n];
  return load_status, initLoading, initTimeSpent, WW;

def gV(w):
  initLoading = w[0:m];
  initTimeSpent = np.abs(w[m:2*m]);
  w = w[2*m:]
  w = np.reshape(w, (m, 3, 6));
  return objV(w, initPos, initTimeSpent, initLoading, distMat, reqLoads);

In [ ]:
gV(pos)

(array([[-10.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   2.,   0.],
        [  0.,   6.,   0.,   0.,   2.]]),
 array([0, 0]),
 array([0.15952907, 0.54368159]),
 array([[[ 0.        ,  0.        ,  2.        ,  2.        ,
           2.        ,  2.        ],
         [-2.        , -3.        ,  2.        ,  3.        ,
           0.        ,  0.        ],
         [ 0.6991146 ,  0.27893253,  0.26154098,  1.72180611,
           0.90913278,  4.7124667 ]],
 
        [[ 0.        ,  0.        ,  2.        ,  1.        ,
           1.        ,  2.        ],
         [-2.        , -3.        ,  1.        ,  0.        ,
           2.        ,  2.        ],
         [ 0.04220646,  0.26191177,  0.03102778,  2.39303425,
           0.36164273,  1.84518334]]]))